# This script implements the analytical formulae developed in KKL03 and Kim, Perera, McLaughlin, 2012 for calculating the merger rate

These were used in the early stages of our analysis to verify our results. The merger rates calculated in the paper use our method of direct convolution of the probability distribution functions.

In [ ]:
import numpy as np, astropy, scipy, glob, sys
import matplotlib.pyplot as pl
from scipy.optimize import curve_fit
from scipy.misc import factorial
import scipy.signal as sig
import scipy.stats as st
from scipy.interpolate import interp1d
import plot_conf_int

#%pylab
%matplotlib inline

In [ ]:
def pdf_two_binaries(rates1, alpha1, t_life_1, f_b_1, rates2, alpha2, t_life_2, f_b_2):
    
    A = alpha1 * t_life_1 / f_b_1
    B = alpha2 * t_life_2 / f_b_2
    
    r_plus = rates1 + rates2
    r_minus = rates1 - rates2
    
    pdf = (A * B / (B - A)) ** 2 * ( r_plus * (np.exp(-1 * A * r_plus) + np.exp(-1 * B * r_plus))
                                   - (2 / (B - A)) * (np.exp(-1 * A * r_plus) - np.exp(-1 * B * r_plus)) )
    
    return r_plus, pdf


In [ ]:
def pdf_three_binaries(rates1, alpha1, t_life_1, f_b_1, rates2, alpha2, t_life_2, f_b_2, rates3, alpha3, t_life_3, f_b_3):
    
    c1 = alpha1 * t_life_1 / f_b_1
    c2 = alpha2 * t_life_2 / f_b_2
    c3 = alpha3 * t_life_3 / f_b_3
    
    r_plus = rates1 + rates2 + rates3
    
    pdf = ( (c1 * c2 * c3) ** 2 / ( (c2 - c1) ** 3 * (c3 - c1) ** 3 * (c3 - c2) ** 3 ) ) * (
    (c3 - c2) ** 3 * np.exp(-1 * c1 * r_plus) * ( -2 * (-2 * c1 + c2 + c3) + r_plus * ( -1 * c1 * (c2 + c3) + (c1 ** 2 + c2 * c3) ) ) + 
    (c3 - c1) ** 3 * np.exp(-1 * c2 * r_plus) * (  2 * (c1 - 2 * c2 + c3) + r_plus * (c2 * (c3 + c1) - (c2 ** 2 + c3 * c1))) +
    (c2 - c1) ** 3 * np.exp(-1 * c3 * r_plus) * ( -2 * (c1 + c2 - 2 * c3) + r_plus * (-1 * c3 * (c1 + c2) + (c3 ** 2 + c1 * c2))) )
    
    return r_plus, pdf


In [ ]:
names = np.array([r'B1913+16', r'B1534+12', r'J1757$-$1854', 'J1946+2052', '1913+1102', r'J0737$-$3039A', 'J1756$-$2251', 'J1906$-$0746'])

N_tots = np.linspace(10, 4e3, 1000)
#Correct alphas
alphas = np.array([0.00356455, 0.00599778, 0.00466759, 0.00442606, 0.00487211, 0.00242033, 0.00604415, 0.01117695])
#decrease mean luminosity alphas
#alphas = np.array([0.00290474, 0.00479277, 0.00302507, 0.002158, 0.00319983, 0.00124934, 0.00424225, 0.00947576])

#Correct n_psr
N_psr = np.array([281, 167, 214, 226, 205, 413, 165, 89])
#decrease mean luminosity n_psr
#N_psr = np.array([344, 209, 331, 463, 313, 800, 236, 106])

#t_life using the GW coalescence time for J1906+0746 as its death time:
#t_life = np.array([0.37e3, 2.93e3, 0.162e3, 0.293e3, 3.125e3, 0.24e3, 1.69e3, 0.3e3])
#t_life using the time for 1906 to cross into the death valley, with eq. 6 and 9 and Dunc's (FIDUCIAL) eq. resp.
#t_life = np.array([0.37e3, 2.93e3, 0.162e3, 0.293e3, 3.125e3, 0.24e3, 1.69e3, 0.003e3])
#t_life = np.array([0.37e3, 2.93e3, 0.162e3, 0.293e3, 3.125e3, 0.24e3, 1.69e3, 0.03e3])
# FIDUCIAL
t_life = np.array([0.37e3, 2.93e3, 0.162e3, 0.293e3, 3.125e3, 0.24e3, 1.69e3, 0.06e3]) 

#Correct f_b
f_b = np.array([5.72, 6.04, 4.59, 4.59, 4.59, 2.00, 4.59, 4.59])
#f_b = np.array([5.72, 6.04, 24, 24, 24, 2.00, 24, 24])
#f_b = np.array([5.72, 6.04, 10, 10, 10, 2.00, 10, 10])

pop_pdfs = get_pop_pdfs(N_tots, alphas)

rates, rate_pdfs = get_rate_pdfs(N_tots, N_psr, alphas, t_life, f_b)

In [ ]:
#1913 and 1534 in this block:

rates_1913_1534, pdf_1913_1534 = pdf_two_binaries(rates[2], alphas[2], t_life[2], f_b[2], rates[1], alphas[1], t_life[1], f_b[1])

print "Predicted galactic merger rate for 1913 + 1534 is {} per Myr".format(rates_1913_1534[np.argmax(pdf_1913_1534)])

#0737 and 1946 in this block:

rates_0737_1946, pdf_0737_1946 = pdf_two_binaries(rates[0], alphas[0], t_life[0], f_b[0], rates[4], alphas[4], t_life[4], f_b[4])

print "Predicted galactic merger rate for 0737 + 1946 is {} per Myr".format(rates_0737_1946[np.argmax(pdf_0737_1946)])